# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [15]:
# review a document in the establishments collection
pprint(db['establishments'].find_one())

{'AddressLine1': '3 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Boodles',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.638929767669,
 'FHRSID': 1349980,
 'LocalAuthorityBusinessID': 'PI/000084064',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2021-06-14T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64159a63ae228f2fc65988ca'),
 'geocode': {'latitude': '51.150879', 'longitude': '1.378536'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1349980',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T00

In [14]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [16]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    'AddressLine1' : 'Greenwich',
    'BusinessName' : "Penang Flavours"
}

In [17]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_flavours)

In [18]:
# Check that the new restaurant was inserted
establishments.find_one(penang_flavours)

{'_id': ObjectId('64159df4831b03eaa7f9d581'),
 'AddressLine1': 'Greenwich',
 'BusinessName': 'Penang Flavours'}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [22]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
r_c_c_businessId = establishments.find_one({"BusinessType" : "Restaurant/Cafe/Canteen"},{"BusinessTypeID" : 1, "BusinessType" : 1})
pprint(r_c_c_businessId)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('64159a63ae228f2fc65988ca')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [24]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"BusinessName": "Penang Flavours"},{'$set' : {'BusinessTypeID' : 1}})

In [27]:
# Confirm that the new restaurant was updated
pprint(list(establishments.find({"BusinessName": "Penang Flavours"})))

[{'AddressLine1': 'Greenwich',
  'BusinessName': 'Penang Flavours',
  'BusinessTypeID': 1,
  '_id': ObjectId('64159df4831b03eaa7f9d581')}]


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [30]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName":"Dover"}
print(establishments.count_documents(query))

994


In [47]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query)

In [54]:
# Check if any remaining documents include Dover
print(establishments.count_documents(query))

0


In [56]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one({}))

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('64159a63ae228f2fc6598baf'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [57]:
# Change the data type from String to Decimal for longitude
establishments.update_many({}, [{'$set' :{"geocode.longitude" : {"$toDouble" : "$geocode.longitude"}}}])

In [58]:
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [{"$set" : {"geocode.latitude" : {"$toDouble" : "$geocode.latitude"}}}])

In [59]:
# Check that the coordinates are now numbers
pprint(establishments.find_one({}, {"geocode":1}))

{'_id': ObjectId('64159a63ae228f2fc6598baf'),
 'geocode': {'latitude': 51.083812, 'longitude': 1.195625}}
